In [1]:
# imports
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from env import get_connection
import os
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
def acquire_curriculum_data():
    
    '''read data from csv file of a local storage if exists. If csv file does not exist, read data from 
    server using SQL, write data into csv file for cache. Return a dataframe
    '''
    
    filename = 'curriculum_logs.csv'
     
    if os.path.exists(filename):
        # read data from csv file of local storage
        df = pd.read_csv(filename)
    else: 
        query = '''
                SELECT *
                FROM logs
                '''

        url = get_connection('curriculum_logs')

        # read data from server using SQL
        df = pd.read_sql(query, url)
        
        # write to data csv file for cache
        df.to_csv('curriculum_logs.csv', index_label=False)
        
    return df
        
        

In [3]:
# acquire data
df = acquire_curriculum_data()

In [4]:
# peak into data
df.head()

,date,time,path,user_id,cohort_id,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


In [5]:
# check for data types
df.dtypes

date          object
time          object
path          object
user_id        int64
cohort_id    float64
ip            object
dtype: object

In [6]:
def prep(df, user):
    '''takes a dataframe and user id as user and return a resmaple data counting the hit on each day of a specific user
    '''
    
    df = df[df.user_id == user]
    # convert data types from object to datetime
    df.date = pd.to_datetime(df.date)
    
    # set data as index
    df = df.set_index(df.date)
    
    # resample the data and count the hit on each day
    pages = df['path'].resample('d').count()
    
    return pages

In [7]:
daily = prep(df, 1)

daily.head()

date
2018-01-26    7
2018-01-27    1
2018-01-28    0
2018-01-29    6
2018-01-30    5
Freq: D, Name: path, dtype: int64

In [8]:
# get average 
weekly_avg = daily.ewm(span=7).mean()
bi_weekly_avg = daily.ewm(span=14).mean()
monthly_avg = daily.ewm(span=30).mean()


In [9]:
# peak into data
weekly_avg.head()

date
2018-01-26    7.000000
2018-01-27    3.571429
2018-01-28    2.027027
2018-01-29    3.480000
2018-01-30    3.978233
Freq: D, Name: path, dtype: float64

In [10]:
# # peak into data
# bi_weekly_avg.head()

In [11]:
# # peak into data
# monthly_avg.head()

In [12]:
# get average
midband = daily.ewm(span=7).mean()

# # peak into data
# midband.head()

In [13]:
# get standard deviation
std = daily.ewm(span=7).std()

# peak into data
# std.head()

In [14]:
# get upper and lower bands
upper_band = weekly_avg + std * 1.5
lower_band = weekly_avg - std * 1.5

In [15]:
# peak into data
# upper_band.head()

In [16]:
# peak into data
# lower_band.head()

In [17]:
def compute_pct_b(pages, span, weight, user):
    '''takes in a series, span, weight and user id.
    return a dataframe with %b, Percent Bandwidth'''
    
    # compute average
    midband = pages.ewm(span=span).mean()
    # compute standard deviation
    stdev = pages.ewm(span=span).std()
    # compute upper bound
    upper_bound = midband + stdev*weight
    # compute lower bond
    lower_bound = midband - stdev*weight
    # concat 
    my_df = pd.concat([pages, midband, upper_bound, lower_bound], axis=1)
    # rename columns
    my_df.columns = ['pages', 'midband', 'upper_bound', 'lower_bound']
    # compute%b, Percent Bandwidth
    my_df['pct_b'] = (my_df['pages'] - my_df['lower_bound'])/(my_df['upper_bound'] - my_df['lower_bound'])
    
    my_df['user_id'] = user
    
    return my_df

In [18]:
# get a dataframe with %b, Percent Bandwidth
compute_pct_b(daily, 14, 1.5,1)

,pages,midband,upper_bound,lower_bound,pct_b,user_id
date,,,,,,
2018-01-26,7,7.000000,NaN,NaN,NaN,1
2018-01-27,1,3.785714,10.149675,-2.578247,0.281134,1
2018-01-28,0,2.339559,7.844148,-3.165031,0.287490,1
2018-01-29,6,3.459391,8.687553,-1.768771,0.742973,1
2018-01-30,5,3.861333,8.363770,-0.641103,0.626450,1
...,...,...,...,...,...,...
2021-04-17,0,11.475254,35.554447,-12.603938,0.261718,1
2021-04-18,8,11.011887,33.503697,-11.479923,0.433045,1
2021-04-19,10,10.876969,31.822549,-10.068611,0.479066,1


In [19]:
def find_anomalies(df, user, span, weight):
    '''takes a datdaframe, user id, span and weight
    return a dataframe with anomalies
    '''
    #  get a resmaple data counting the hit on each day of a specific userv
    pages = prep(df, user)
    
    # get a dataframe with %b, Percent Bandwidth''
    my_df = compute_pct_b(pages, span, weight, user)
    
    # return a dataframe with anomalies
    return my_df[my_df.pct_b>1]

In [20]:
# compute anamolies
find_anomalies(df, 1,14, 1.5)

,pages,midband,upper_bound,lower_bound,pct_b,user_id
date,,,,,,
2018-02-07,13,6.058634,12.430064,-0.312797,1.044726,1
2018-02-20,54,8.669546,37.136861,-19.797770,1.296184,1
2018-03-19,35,10.168935,29.809882,-9.472012,1.132125,1
2018-03-26,31,10.652715,29.650432,-8.345001,1.035519,1
2018-04-16,20,5.442606,17.594023,-6.708812,1.099000,1
...,...,...,...,...,...,...
2021-02-21,35,7.167490,26.276843,-11.941862,1.228243,1
2021-03-25,25,7.761717,20.199175,-4.675740,1.192999,1
2021-04-12,22,5.000222,16.751176,-6.750733,1.223336,1
